In [1]:
# same code as above but storing image paths in lists for direct access

import os
import re
from collections import defaultdict

# Define the path to the folder containing the images
image_folder_path = 'Analysis/bilawal_dhamaar'

# Define the section-wise row numbers
articulation_rows = [12, 14, 31, 33]
kann_swar_rows = [4, 10, 16, 22, 29]
swar_rows = [5, 7, 11, 18, 23, 26, 30]
lyrics_rows = [6, 8, 13, 19, 24, 27, 32]

# Define the subgroup ranges
subgroup_ranges = [
    (4, 7),
    (7, 8),
    (10, 14),
    (16, 19),
    (22, 24),
    (26, 27),
    (29, 33)
]

# Define the beat count (size of the lists)
beat_count = 14

# Function to extract information from the image filename
def extract_info_from_filename(filename):
    pattern = r'(\d+)_row(\d+)(?:_col(\d+))?_x(\d+)_y(\d+)_w(\d+)_h(\d+)'
    match = re.match(pattern, filename)
    if match:
        page_num = int(match.group(1))
        row_num = int(match.group(2))
        col_num = int(match.group(3)) if match.group(3) else None
        x = int(match.group(4))
        y = int(match.group(5))
        width = int(match.group(6))
        height = int(match.group(7))
        # Use os.path.join to handle path separators correctly
        image_path = os.path.normpath(os.path.join(image_folder_path, filename))
        return page_num, row_num, col_num, x, y, width, height, image_path
    return None

# Load all image filenames and extract their information
image_files = os.listdir(image_folder_path)
image_info = [extract_info_from_filename(f) for f in image_files]
image_info = [info for info in image_info if info is not None]

# Organize images by row and column
row_col_images = defaultdict(lambda: defaultdict(list))
for info in image_info:
    page_num, row_num, col_num, x, y, width, height, image_path = info
    row_col_images[row_num][col_num].append((x, y, width, height, image_path))

# Function to pad lists to match the beat count
def pad_lists(lists, size):
    if len(lists) < size:
        padding = [[] for _ in range(size - len(lists))]
        return padding + lists
    return lists

# Function to process a subgroup and create the lists of lists
def process_subgroup(subgroup_range, is_first_subgroup):
    start_row, end_row = subgroup_range
    
    # Find the swar row in this subgroup
    swar_row = None
    for row in swar_rows:
        if start_row <= row <= end_row:
            swar_row = row
            break
    
    if not swar_row:
        return None, None, None, None, None
    
    # Find the kann swar row in this subgroup
    kann_swar_row = None
    for row in kann_swar_rows:
        if start_row <= row <= end_row:
            kann_swar_row = row
            break
    
    # Find the articulation rows in this subgroup
    articulation_rows_in_subgroup = [row for row in articulation_rows if start_row <= row <= end_row]
    
    # Find the lyrics row in this subgroup
    lyrics_row = None
    for row in lyrics_rows:
        if start_row <= row <= end_row:
            lyrics_row = row
            break
    
    # Get the swar images and their column numbers
    swar_images = row_col_images[swar_row]
    swar_cols = sorted(swar_images.keys())
    
    # Get the kann swar images and their column numbers (if kann swar row exists)
    kann_swar_images = row_col_images[kann_swar_row] if kann_swar_row else {}
    kann_swar_cols = sorted(kann_swar_images.keys())
    
    # Get the lyrics images (if lyrics row exists)
    lyrics_images = row_col_images[lyrics_row] if lyrics_row else {}
    lyrics_cols = sorted(lyrics_images.keys()) if lyrics_row else []
    
    # Create the lists of lists
    swar_list = []
    kann_swar_list = []
    swar_articulation_checks = [False] * len(swar_cols)
    lyrics_articulation_checks = [False] * len(lyrics_cols)
    lyrics_list = []
    
    # Case 1: If there is an explicit kann swar row
    if kann_swar_row:
        swar_index = 0
        kann_swar_index = 0
        
        while swar_index < len(swar_cols) or kann_swar_index < len(kann_swar_cols):
            swar_col = swar_cols[swar_index] if swar_index < len(swar_cols) else None
            kann_swar_col = kann_swar_cols[kann_swar_index] if kann_swar_index < len(kann_swar_cols) else None
            
            # If both columns exist and match
            if swar_col is not None and kann_swar_col is not None and swar_col == kann_swar_col:
                swar_list.append([x[4] for x in swar_images[swar_col]])  # Store image paths
                kann_swar_list.append([x[4] for x in kann_swar_images[kann_swar_col]])  # Store image paths
                swar_index += 1
                kann_swar_index += 1
            # If swar column exists but kann swar column doesn't match or is missing
            elif swar_col is not None and (kann_swar_col is None or swar_col < kann_swar_col):
                swar_list.append([x[4] for x in swar_images[swar_col]])  # Store image paths
                kann_swar_list.append([])
                swar_index += 1
            # If kann swar column exists but swar column doesn't match or is missing
            elif kann_swar_col is not None and (swar_col is None or kann_swar_col < swar_col):
                # Assign the kann swar to the next available swar column
                if swar_index < len(swar_cols):
                    swar_list.append([x[4] for x in swar_images[swar_cols[swar_index]]])  # Store image paths
                    kann_swar_list.append([x[4] for x in kann_swar_images[kann_swar_col]])  # Store image paths
                    swar_index += 1
                    kann_swar_index += 1
                else:
                    # If no more swar columns are available, append an empty list
                    swar_list.append([])
                    kann_swar_list.append([x[4] for x in kann_swar_images[kann_swar_col]])  # Store image paths
                    kann_swar_index += 1
    
    # Case 2: If there is no explicit kann swar row, check for hidden kann swars in the swar row
    else:
        for col in swar_cols:
            images_in_col = swar_images[col]
            if len(images_in_col) == 1:
                # Only one image in this column, so no hidden kann swar
                swar_list.append([images_in_col[0][4]])  # Store image path
                kann_swar_list.append([])
            else:
                # Multiple images in the same column, so identify hidden kann swars
                # Sort images by y-value (lower y-value is kann swar)
                sorted_images = sorted(images_in_col, key=lambda x: x[1])  # Sort by y-value
                kann_swar_list.append([sorted_images[0][4]])  # Lower y-value is kann swar (store image path)
                swar_list.append([sorted_images[1][4]])  # Higher y-value is swar (store image path)
    
    # Handle articulation rows
    for articulation_row in articulation_rows_in_subgroup:
        # Find the row just before the articulation row
        prev_row = articulation_row - 1
        if prev_row in swar_rows:
            # Swar articulation
            articulation_images = row_col_images[articulation_row]
            articulation_cols = sorted(articulation_images.keys())
            for i, col in enumerate(swar_cols):
                if col in articulation_cols:
                    swar_articulation_checks[i] = True
        elif prev_row in lyrics_rows:
            # Lyrics articulation
            articulation_images = row_col_images[articulation_row]
            articulation_cols = sorted(articulation_images.keys())
            for i, col in enumerate(swar_cols):
                if col in articulation_cols:
                    lyrics_articulation_checks[i] = True
    
    # Handle lyrics row (append images one by one without comparing column numbers)
    if lyrics_row:
        # Get all lyrics images in order
        lyrics_cols = sorted(lyrics_images.keys())
        for col in lyrics_cols:
            lyrics_list.append([x[4] for x in lyrics_images[col]])  # Store image paths
    else:
        lyrics_list = [[] for _ in range(len(swar_cols))]
    
    # Pad lists to match the beat count
    if is_first_subgroup:
        swar_list = pad_lists(swar_list, beat_count)
        kann_swar_list = pad_lists(kann_swar_list, beat_count)
        swar_articulation_checks = pad_lists(swar_articulation_checks, beat_count)
        lyrics_articulation_checks = pad_lists(lyrics_articulation_checks, beat_count)
        lyrics_list = pad_lists(lyrics_list, beat_count)
    else:
        if len(swar_list) < beat_count:
            swar_list += [[] for _ in range(beat_count - len(swar_list))]
        if len(kann_swar_list) < beat_count:
            kann_swar_list += [[] for _ in range(beat_count - len(kann_swar_list))]
        if len(swar_articulation_checks) < beat_count:
            swar_articulation_checks += [False for _ in range(beat_count - len(swar_articulation_checks))]
        if len(lyrics_articulation_checks) < beat_count:
            lyrics_articulation_checks += [False for _ in range(beat_count - len(lyrics_articulation_checks))]
        if len(lyrics_list) < beat_count:
            lyrics_list += [[] for _ in range(beat_count - len(lyrics_list))]
    
    return swar_list, kann_swar_list, swar_articulation_checks, lyrics_articulation_checks, lyrics_list

# Process each subgroup and store the results
subgroup_results = {}
for i, subgroup_range in enumerate(subgroup_ranges):
    is_first_subgroup = (i == 0)
    swar_list, kann_swar_list, swar_articulation_checks, lyrics_articulation_checks, lyrics_list = process_subgroup(subgroup_range, is_first_subgroup)
    if swar_list and kann_swar_list:
        subgroup_results[subgroup_range] = {
            'swar_list': swar_list,
            'kann_swar_list': kann_swar_list,
            'swar_articulation_checks': swar_articulation_checks,
            'lyrics_articulation_checks': lyrics_articulation_checks,
            'lyrics_list': lyrics_list
        }

# Print the results for each subgroup
for subgroup_range, results in subgroup_results.items():
    print(f"Subgroup Range: {subgroup_range}")
    print(f"Kann Swar List: {results['kann_swar_list']}")
    print(f"Swar List: {results['swar_list']}")
    print(f"Swar Articulation Checks: {results['swar_articulation_checks']}")
    print(f"Lyrics List: {results['lyrics_list']}")
    print(f"Lyrics Articulation Checks: {results['lyrics_articulation_checks']}")
    print("-" * 80)

Subgroup Range: (4, 7)
Kann Swar List: [[], [], [], [], [], [], [], [], [], [], [], ['Analysis\\bilawal_dhamaar\\0_row4_col12_x400_y145_w7_h10.png'], ['Analysis\\bilawal_dhamaar\\0_row4_col13_x434_y145_w9_h10.png'], []]
Swar List: [[], [], [], [], [], [], [], [], [], [], [], ['Analysis\\bilawal_dhamaar\\0_row5_col12_x406_y162_w10_h12.png'], ['Analysis\\bilawal_dhamaar\\0_row5_col13_x439_y162_w10_h14.png'], ['Analysis\\bilawal_dhamaar\\0_row5_col14_x472_y166_w9_h5.png']]
Swar Articulation Checks: [[], [], [], [], [], [], [], [], [], [], [], False, False, False]
Lyrics List: [[], [], [], [], [], [], [], [], [], [], [], ['Analysis\\bilawal_dhamaar\\0_row6_col12_x405_y188_w11_h15.png'], ['Analysis\\bilawal_dhamaar\\0_row6_col13_x436_y188_w14_h12.png'], ['Analysis\\bilawal_dhamaar\\0_row6_col14_x473_y187_w8_h12.png']]
Lyrics Articulation Checks: [[], [], [], [], [], [], [], [], [], [], [], False, False, False]
--------------------------------------------------------------------------------


In [2]:
import os
import cv2
import numpy as np
from skimage.morphology import binary_erosion, binary_dilation, square
from skimage import img_as_ubyte

# Define the path to the folder to store segmented images
segmented_folder_path = os.path.normpath('Analysis/bilawal_dhamaar_segmented')
os.makedirs(segmented_folder_path, exist_ok=True)

# Function to preprocess an image
def preprocess_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 21, 5)
    return thresh

# Function to separate articulation in an image
def separate_articulation(image):
    processed_image = preprocess_image(image)
    contours, _ = cv2.findContours(processed_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if 10 < h < 21 and w > 25:
            upper_part = image[:y, :]
            if upper_part.shape[0] > 0:
                return upper_part, True  # Return the upper part and a flag indicating segmentation was successful
            break
    
    return image, False  # Return the original image and a flag indicating no segmentation

# Function to segment a word into multiple images
def segment_image(img):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply simple binary thresholding and invert the image
    _, binary = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

    # Define structuring elements
    structuring_element2 = np.ones((2, 2), dtype=bool)
    structuring_element_erosion = square(3)

    # Apply binary dilation to fill gaps
    dilated = binary_dilation(binary, footprint=structuring_element2)

    # Apply binary erosion to separate connected components
    eroded = binary_erosion(dilated, footprint=structuring_element_erosion)
    eroded = img_as_ubyte(eroded)  # Convert to uint8 for display purposes

    # Perform vertical projection to find potential cut lines
    vertical_projection = np.sum(eroded, axis=0)

    # Find cut points by identifying valleys in the projection with heuristic
    threshold = 0.15 * np.max(vertical_projection)
    valleys = [x for x, y in enumerate(vertical_projection) if y < threshold]

    # Apply heuristic: if two consecutive valleys are close, take the right one
    cut_points = []
    min_distance = 13
    i = 0
    while i < len(valleys) - 1:
        if (valleys[i + 1] - valleys[i]) < min_distance:
            cut_points.append(valleys[i + 1])
            i += 2  # Skip the next valley since we took the right one
        else:
            cut_points.append(valleys[i])
            i += 1
    if i == len(valleys) - 1:
        cut_points.append(valleys[i])  # Add the last valley if it's not processed

    # Ensure no duplicate cut points and sort them
    cut_points = sorted(set(cut_points))

    # Separate the image at cut points
    cut_images = []
    start = 0
    for cut_point in cut_points:
        if cut_point - start > 10:  # Ensure segments are large enough
            cut_image = img[:, start:cut_point]
            cut_images.append(cut_image)
            start = cut_point

    # Add the last segment
    cut_images.append(img[:, start:])

    return cut_images

# Function to merge segments based on height-to-width ratio
def merge_segments(segments):
    final_images = []
    i = 0
    while i < len(segments):
        current_image = segments[i]
        current_ratio = current_image.shape[0] / current_image.shape[1]

        ratio_threshold = 1.8

        if current_image.shape[0] > 35:
            ratio_threshold = 2.9
        
        # If the ratio is greater than the threshold and it's the first segment
        if current_ratio > ratio_threshold and i == 0:
            # Merge with the next segment
            if i + 1 < len(segments):
                current_image = np.hstack((current_image, segments[i + 1]))
                final_images.append(current_image)
                i += 2
            else:
                final_images.append(current_image)
                i += 1
        # If two or more consecutive segments have a ratio greater than the threshold
        elif i < len(segments) - 1 and (segments[i + 1].shape[0] / segments[i + 1].shape[1]) > ratio_threshold:
            while i < len(segments) - 1 and (segments[i + 1].shape[0] / segments[i + 1].shape[1]) > ratio_threshold:
                current_image = np.hstack((current_image, segments[i + 1]))
                i += 1
            final_images.append(current_image)
            i += 1
        # If the ratio is greater than the threshold and it's not the first segment
        elif current_ratio > ratio_threshold and i != 0:
            # Merge with the previous segment
            if final_images:
                final_images[-1] = np.hstack((final_images[-1], current_image))
            else:
                final_images.append(current_image)
            i += 1
        else:
            final_images.append(current_image)
            i += 1

    return final_images

# Function to process a single image, segment, and save the results in the provided folder
def segment_word(image_path, output_folder):
    # Load the image
    img = cv2.imread(image_path)
    if img is None:
        return []
    
    # Segment the image
    segmented_images = segment_image(img)
    
    # Merge segments based on height-to-width ratio
    final_images = merge_segments(segmented_images)
    
    # Save the segmented images
    image_base_name = os.path.splitext(os.path.basename(image_path))[0]
    segmented_paths = []
    for i, segmented_image in enumerate(final_images):
        seg_image_path = os.path.normpath(os.path.join(output_folder, f'{image_base_name}_seg{i+1}.png'))
        cv2.imwrite(seg_image_path, segmented_image)
        segmented_paths.append(seg_image_path)
    
    return segmented_paths

# Function to update lists based on segmentation
def update_lists_with_segmentation(subgroup_results):
    for subgroup_range, results in subgroup_results.items():
        swar_list = results['swar_list']
        lyrics_list = results['lyrics_list']
        swar_articulation_checks = results['swar_articulation_checks']
        lyrics_articulation_checks = results['lyrics_articulation_checks']
        
        # Apply articulation segmentation to swar row
        for i in range(len(swar_list)):
            if not swar_articulation_checks[i] and swar_list[i]:  # Check if articulation is False and the list is not empty
                swar_image_path = swar_list[i][0]  # Get the image path
                swar_image = cv2.imread(swar_image_path)  # Load the image
                if swar_image is not None:
                    segmented_image, is_segmented = separate_articulation(swar_image)
                    if is_segmented:
                        swar_articulation_checks[i] = True  # Update articulation check
                        # Save the segmented image with the original name
                        original_name = os.path.basename(swar_image_path)
                        seg_image_path = os.path.normpath(os.path.join(segmented_folder_path, original_name))
                        cv2.imwrite(seg_image_path, segmented_image)
                        swar_list[i] = [seg_image_path]  # Update the list with the new image path
        
        # Apply articulation segmentation to lyrics row
        for i in range(len(lyrics_list)):
            if not lyrics_articulation_checks[i] and lyrics_list[i]:  # Check if articulation is False and the list is not empty
                lyrics_image_path = lyrics_list[i][0]  # Get the image path
                lyrics_image = cv2.imread(lyrics_image_path)  # Load the image
                if lyrics_image is not None:
                    segmented_image, is_segmented = separate_articulation(lyrics_image)
                    if is_segmented:
                        lyrics_articulation_checks[i] = True  # Update articulation check
                        # Save the segmented image with the original name
                        original_name = os.path.basename(lyrics_image_path)
                        seg_image_path = os.path.normpath(os.path.join(segmented_folder_path, original_name))
                        cv2.imwrite(seg_image_path, segmented_image)
                        lyrics_list[i] = [seg_image_path]  # Update the list with the new image path
        
        # Apply word segmentation to swar row
        for i in range(len(swar_list)):
            if swar_articulation_checks[i] and swar_list[i]:  # Check if articulation is True and the list is not empty
                swar_image_path = swar_list[i][0]  # Get the image path
                segmented_paths = segment_word(swar_image_path, segmented_folder_path)
                if segmented_paths:
                    swar_list[i] = segmented_paths  # Update the list with segmented image paths
        
        # Apply word segmentation to lyrics row
        for i in range(len(lyrics_list)):
            if lyrics_articulation_checks[i] and lyrics_list[i]:  # Check if articulation is True and the list is not empty
                lyrics_image_path = lyrics_list[i][0]  # Get the image path
                segmented_paths = segment_word(lyrics_image_path, segmented_folder_path)
                if segmented_paths:
                    lyrics_list[i] = segmented_paths  # Update the list with segmented image paths
        
        # Update the results
        subgroup_results[subgroup_range]['swar_list'] = swar_list
        subgroup_results[subgroup_range]['lyrics_list'] = lyrics_list
        subgroup_results[subgroup_range]['swar_articulation_checks'] = swar_articulation_checks
        subgroup_results[subgroup_range]['lyrics_articulation_checks'] = lyrics_articulation_checks

# Example usage
update_lists_with_segmentation(subgroup_results)

# Print the updated results for each subgroup
for subgroup_range, results in subgroup_results.items():
    print(f"Subgroup Range: {subgroup_range}")
    print(f"Kann Swar List: {results['kann_swar_list']}")
    print(f"Swar List: {results['swar_list']}")
    print(f"Lyrics List: {results['lyrics_list']}")
    print(f"Swar Articulation Checks: {results['swar_articulation_checks']}")
    print(f"Lyrics Articulation Checks: {results['lyrics_articulation_checks']}")
    print("-" * 80)

Subgroup Range: (4, 7)
Kann Swar List: [[], [], [], [], [], [], [], [], [], [], [], ['Analysis\\bilawal_dhamaar\\0_row4_col12_x400_y145_w7_h10.png'], ['Analysis\\bilawal_dhamaar\\0_row4_col13_x434_y145_w9_h10.png'], []]
Swar List: [[], [], [], [], [], [], [], [], [], [], [], ['Analysis\\bilawal_dhamaar\\0_row5_col12_x406_y162_w10_h12.png'], ['Analysis\\bilawal_dhamaar\\0_row5_col13_x439_y162_w10_h14.png'], ['Analysis\\bilawal_dhamaar\\0_row5_col14_x472_y166_w9_h5.png']]
Lyrics List: [[], [], [], [], [], [], [], [], [], [], [], ['Analysis\\bilawal_dhamaar\\0_row6_col12_x405_y188_w11_h15.png'], ['Analysis\\bilawal_dhamaar\\0_row6_col13_x436_y188_w14_h12.png'], ['Analysis\\bilawal_dhamaar\\0_row6_col14_x473_y187_w8_h12.png']]
Swar Articulation Checks: [[], [], [], [], [], [], [], [], [], [], [], False, False, False]
Lyrics Articulation Checks: [[], [], [], [], [], [], [], [], [], [], [], False, False, False]
--------------------------------------------------------------------------------


In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('cnn_recognizer_music_15_v1.h5')

# Preprocess the input image
def preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if image is None:
        raise ValueError(f"Unable to read image at path: {image_path}")
    image = cv2.resize(image, (32, 32))  # Resize to match the model's input size
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Pass the image through the model and get predictions
def predict_class(image_path):
    preprocessed_image = preprocess_image(image_path)
    predictions = model.predict(preprocessed_image)
    predicted_class_index = np.argmax(predictions, axis=1)
    max_probability = np.max(predictions, axis=1)
    return predicted_class_index[0], max_probability[0]

# Define the classes
# classes = ["सा", "रे", "ग", "म", "प", "ध", "नि", "रे॒", "ग॒", "ध॒", "नि॒", "म॑", 
#            "सा\u0951", "रे\u0951", "ग\u0951", "म\u0951", "प\u0951", "ध\u0951", "रे॒\u0951", "ग॒\u0951", "म॑'", 
#            "म\u093C", "म॑\u093C", "प\u093C", "ध॒\u093C", "ध\u093C", "नि॒\u093C", "नि\u093C", 
#            ")", ",", "-", "४", "O", "(", "^^", "X", "३", "२", "|", "<_>"]

classes = ["saa", "re", "ga", "ma", "pa", "dha", "ni", "re-", "ga-", "dha-", "ni-", "ma#", 
           "saa'", "re'", "ga'", "ma'", "pa'", "dha'", "re-'", "ga-'", "ma#'", 
           "ma,", "ma#,", "pa,", "dha-,", "dha,", "ni-,", "ni,", 
           ")", ",", "-", "४", "O", "(", "^^", "X", "३", "२", "|", "<_>"]

classes = ["c", "d", "e", "f", "g", "a", "b", "d-", "e-", "a-", "b-", "f#", 
           "cc", "dd", "ee", "ff", "gg", "aa", "dd-", "ee-", "ff#", 
           "F", "F#", "G", "A-", "A", "B-", "B", 
           ")", ",", "-", "४", "O", "(", "^^", "X", "३", "२", "|", "<_>"]

# Function to generate new lists with predicted class names
def generate_predicted_lists(subgroup_results):
    predicted_results = {}
    
    for subgroup_range, results in subgroup_results.items():
        # Initialize new lists for predicted class names
        predicted_swar_list = []
        predicted_kann_swar_list = []  # Add this if you have kann_swar_list
        
        # Predict class names for swar_list
        for image_paths in results['swar_list']:
            if image_paths:  # Check if the list is not empty
                predicted_classes = []
                for image_path in image_paths:
                    predicted_class_index, _ = predict_class(image_path)
                    predicted_class_name = classes[predicted_class_index]
                    predicted_classes.append(predicted_class_name)
                predicted_swar_list.append(predicted_classes)
            else:
                predicted_swar_list.append([])  # Append empty list for empty entries
        
        # Predict class names for kann_swar_list (if applicable)
        for image_paths in results['kann_swar_list']:
            if image_paths:  # Check if the list is not empty
                predicted_classes = []
                for image_path in image_paths:
                    predicted_class_index, _ = predict_class(image_path)
                    predicted_class_name = classes[predicted_class_index]
                    predicted_classes.append(predicted_class_name)
                predicted_kann_swar_list.append(predicted_classes)
            else:
                predicted_kann_swar_list.append([])  # Append empty list for empty entries
        
        # Store the predicted results for this subgroup
        predicted_results[subgroup_range] = {
            'predicted_swar_list': predicted_swar_list,
            'predicted_kann_swar_list': predicted_kann_swar_list  
        }
    
    return predicted_results

# Example usage
# Assuming subgroup_results is the dictionary you provided
predicted_results = generate_predicted_lists(subgroup_results)

# Print the predicted results
for subgroup_range, results in predicted_results.items():
    print(f"Subgroup Range: {subgroup_range}")
    print(f"Predicted Swar List: {results['predicted_swar_list']}")
    print(f"Predicted Kann Swar List: {results['predicted_kann_swar_list']}") 
    print("-" * 80)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 573ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━

In [4]:
def predict_sam_taalis_rows(row_col_images, rows_to_predict):
    """
    Predicts the class names for specific rows (sam and taalis) and returns a dictionary
    where each key is a row number and the value is a list of predicted class names.
    """
    predicted_results = {}

    for row_num in rows_to_predict:
        if row_num in row_col_images:  # Check if the row exists in the dictionary
            # Initialize a list to store predicted class names for this row
            predicted_classes = []

            # Get the columns for this row
            columns = sorted(row_col_images[row_num].keys())

            # Loop through each column in the row
            for col_num in columns:
                # Get the list of images for this row and column
                images = row_col_images[row_num][col_num]

                # Predict the class for each image in the column
                for image in images:
                    _, _, _, _, image_path = image  # Extract the image path
                    predicted_class_index, _ = predict_class(image_path)  # Predict the class
                    predicted_class_name = classes[predicted_class_index]  # Get the class name
                    predicted_classes.append(predicted_class_name)

            # Store the predicted results for this row
            predicted_results[row_num] = predicted_classes
        else:
            # If the row doesn't exist, store an empty list
            predicted_results[row_num] = []

    return predicted_results

# Define the rows to predict
sam_taalis_rows = [9, 15, 20, 25, 28, 34]

# Predict the rows
sam_taalis_predictions = predict_sam_taalis_rows(row_col_images, sam_taalis_rows)

# Print the predicted results
for row_num, predictions in sam_taalis_predictions.items():
    print(f"Row {row_num} Predictions: {predictions}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Row 9 Predictions: ['३', 'X', '२', 'O']
Row 15 Predictions: ['३', 'X', '२', 'O']
Row 20 Predictions: ['२', 'X', '२', '२']
Row 